In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
import json, datetime

In [2]:
root_path = 'h:/내 드라이브/Inventory_Order_update/'

with open(root_path+'appdata/db_auth.json') as f:
        temp = json.load(f)
        server = temp['server']
        database = temp['database'] 
        username = temp['username'] 
        password = temp['password']

connection_string = 'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [3]:
query = f"""
SELECT * FROM dbo.supplier
"""

with engine.connect() as conn, conn.begin():  
    sup_list = pd.read_sql(query, conn)

In [4]:
# sup_list[sup_list['SupplierName'].str.startswith('SUN')]
sup_list

,Country,HQID,LastUpdated,State,ID,SupplierName,ContactName,Address1,Address2,City,...,CustomNumber3,CustomNumber4,CustomNumber5,CustomDate1,CustomDate2,CustomDate3,CustomDate4,CustomDate5,Notes,Terms
0,,0,2022-12-14 12:48:41.927,,2082,ONLINE SHIPPING,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
1,,0,2022-12-13 18:06:28.333,,2083,7 MILE SPORTWEAR,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
2,,0,2022-12-13 18:06:04.833,,2084,7 MILE DOWNTOWN,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
3,,0,2022-12-13 18:06:15.103,,2085,UNKNOWN,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
4,,0,2022-12-13 18:04:21.467,,2086,MISC,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,,0,2022-12-13 18:00:38.817,,2147,MISS LASHES TRADING INC.,,,,,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
66,,0,2022-12-13 18:03:25.263,IL,2148,JTS INTERNATIONAL CORP,,408 WOODBURY RD.,,BUFFALO GROVE,...,0.0,0.0,0.0,NaT,NaT,NaT,NaT,NaT,,
67,,0,2022-08-26 17:48:20.563,,2149,No Supplier,,,,,...,1.0,1.0,1.0,2022-08-26 17:48:20.563,2022-08-26 17:48:20.563,2022-08-26 17:48:20.563,2022-08-26 17:48:20.563,2022-08-26 17:48:20.563,,
68,,0,2022-10-04 12:41:27.937,,2150,RED BEAUTY,,,,,...,1.0,1.0,1.0,2022-10-04 12:41:27.937,2022-10-04 12:41:27.937,2022-10-04 12:41:27.937,2022-10-04 12:41:27.937,2022-10-04 12:41:27.937,,


In [9]:
# query = f"""
# SELECT 
#     item.ItemLookupCode, 
#     sp.SupplierID, 
#     dbo.Supplier.SupplierName, 
#     dbo.Supplier.Code, 
#     item.Description as Description,
#     item.Price Price, 
#     item.Cost itemCost, 
#     sp.Cost spCost,
#     dbo.PurchaseOrderEntry.Price recentPrice,
#     dbo.PurchaseOrderEntry.LastUpdated
# FROM 
#     dbo.Item item
# LEFT JOIN 
#     dbo.SupplierList sp ON item.ID=sp.ItemID
# LEFT JOIN 
#     dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
# LEFT JOIN
#     dbo.PurchaseOrderEntry ON item.ID=dbo.PurchaseOrderEntry.ItemID
# WHERE 
#     item.DepartmentID IN (2,4,6)
#     AND item.Inactive = 0
#     AND dbo.PurchaseOrderEntry.LastUpdated >= '2023-8-1'
# ORDER BY 
#     item.SupplierID,
#     item.Description;
# """

query = f"""
SELECT 
    item.ItemLookupCode, 
    item.Quantity,
    sp.SupplierID, 
    dbo.Supplier.SupplierName, 
    dbo.Supplier.Code, 
    item.Description as Description,
    item.Price Price, 
    item.Cost itemCost, 
    sp.Cost spCost
FROM 
    dbo.Item item
LEFT JOIN 
    dbo.SupplierList sp ON item.ID=sp.ItemID
LEFT JOIN 
    dbo.Supplier ON sp.SupplierID=dbo.Supplier.ID
WHERE 
    item.DepartmentID IN (2,4,6)
    AND item.Inactive = 0
ORDER BY 
    item.SupplierID,
    item.Description;
"""

with engine.connect() as conn, conn.begin():  
    fromPOS = pd.read_sql(query, conn, dtype={'ItemLookupCode': str})

In [25]:
fromPOS['Cost']= fromPOS[['itemCost', 'spCost']].apply(max, axis=1)
fromPOS

,ItemLookupCode,SupplierID,SupplierName,Code,Description,Price,itemCost,spCost,recentPrice,LastUpdated,Cost
0,660014310535,2089.0,ALICIA INERNATIONAL INC,10007,10030 JAMIE 1,39.99,19.95,19.95,19.95,2023-05-29 11:48:07.597,19.95
1,660014310559,2089.0,ALICIA INERNATIONAL INC,10007,10030 JAMIE 2,39.99,19.95,19.95,19.95,2023-06-22 16:49:15.347,19.95
2,660014313642,2089.0,ALICIA INERNATIONAL INC,10007,10169 ELEN 3T44,44.99,19.95,19.95,19.95,2023-12-08 17:50:09.990,19.95
3,660014313642,2089.0,ALICIA INERNATIONAL INC,10007,10169 ELEN 3T44,44.99,19.95,19.95,19.95,2023-12-08 12:11:12.093,19.95
4,660014314397,2089.0,ALICIA INERNATIONAL INC,10007,10355 SASSY 34,39.99,19.95,19.95,19.95,2023-12-08 12:11:12.093,19.95
...,...,...,...,...,...,...,...,...,...,...,...
31179,181201419263,2148.0,JTS INTERNATIONAL CORP,10182,PW1ST26 NATURAL,349.99,159.00,159.00,159.00,2023-09-21 13:56:50.190,159.00
31180,181201419287,2148.0,JTS INTERNATIONAL CORP,10182,PW1ST28 NATURAL,379.99,174.00,174.00,174.00,2023-09-21 13:56:50.190,174.00
31181,181201419300,2148.0,JTS INTERNATIONAL CORP,10182,PW1ST30 NATURAL,399.99,194.00,194.00,194.00,2023-09-21 13:56:50.190,194.00
31182,181201419324,2148.0,JTS INTERNATIONAL CORP,10182,PW1ST32 NATURAL,499.99,234.00,234.00,234.00,2023-09-21 13:56:50.190,234.00


In [10]:
temp = fromPOS[fromPOS['itemCost']!=fromPOS['spCost']]

In [10]:
sng_inv = pd.read_excel(root_path+'inv_data\SNG_inv.xlsx', dtype={'Barcode':str})

In [13]:
sng_cost = sng_inv.merge(fromPOS[['ItemLookupCode', 'Cost']], how='left', left_on='Barcode', right_on='ItemLookupCode')

In [ ]:
low_price = fromPOS[fromPOS['Price']<fromPOS['Cost']]

In [14]:
fromPOS = fromPOS[['ItemLookupCode', 'Description','Price', 'Cost']]

In [ ]:
fromPOS

In [ ]:
all_amazon = pd.read_csv(root_path+'inv_data/Amazon_All+Listings+Report.txt', sep='\t', dtype={'product-id': str})
all_amazon = all_amazon[['seller-sku', 'asin1', 'listing-id',
       'price', 'product-id-type', 'product-id', 'status']]

In [ ]:
test = fromPOS.merge(all_amazon, how='left', left_on='ItemLookupCode', right_on='product-id')

In [15]:
fromPOS.to_excel('test.xlsx')

In [ ]:
test

In [ ]:
test.dropna()

In [ ]:
dup = test[test.duplicated('ItemLookupCode')]

In [ ]:
test.to_excel('test.xlsx', index=False)